In [1]:
import WECGrid

engine = WECGrid.Engine("./IEEE_30_bus.RAW")

engine.use(["psse"])


    Sets PSSE environment to latest minor version among installed versions of PSSE 35.
    Use psse35.set_minor(n) to set PSSE35 minor version (n) to use.
        Example, for PSSE 35.0.x, use this as: psse35.set_minor(0)

 PSS(R)E Xplore Version 35
 Copyright (c) 1976-2025
 Siemens Industry, Inc.,
 Power Technologies International                            (PTI)
 This program is a confidential  unpublished  work  created  and  first
 licensed in 1976.  It is a trade secret which is the property of  PTI.
 All use,  disclosure,  and/or reproduction not specifically authorized
 by  PTI  is prohibited.   This  program is protected  under  copyright
 laws  of  non-U.S.  countries  and  by  application  of  international
 treaties.  All  Rights  Reserved  Under  The  Copyright  Laws.


           SIEMENS POWER TECHNOLOGIES INTERNATIONAL

    50 BUS POWER SYSTEM SIMULATOR--PSS(R)E Xplore-35.3.2

             INITIATED ON SUN, MAY 11 2025  18:13
Initialized: PSSE with case IEEE_30_bus.RAW


In [3]:
config = {
    "simLength": 60 * 60 * 12,  # Simulation length of 12 hours
    "Tsample": 300,            # Sampling time of 5 minutes
    "waveHeight": 2.5,         # Wave height in meters
    "wavePeriod": 8,           # Wave period in seconds
}
engine.create_wec(ID=60, model="RM3", farm_size=8, ibus=31, jbus=1, mbase= 1.0, config=config) 


AttributeError: 'PSSEInterface' object has no attribute 'add_wec'

In [1]:
from WecGrid import WecGrid

wec_grid = WecGrid(case="./IEEE_30_bus.RAW")
wec_grid.initialize_psse()
wec_grid.initialize_pypsa()

config = {
    "simLength": 60 * 60 * 12,  # Simulation length of 12 hours
    "Tsample": 300,            # Sampling time of 5 minutes
    "waveHeight": 2.5,         # Wave height in meters
    "wavePeriod": 8,           # Wave period in seconds
}
wec_grid.create_wec(ID=60, model="RM3", farm_size=8, ibus=31, jbus=1, mbase= 1.0, config=config) 

wec_grid.psseObj.simulate(load_curve=True)

wec_grid.pypsaObj.simulate(load_curve=True)

    Sets PSSE environment to latest minor version among installed versions of PSSE 35.
    Use psse35.set_minor(n) to set PSSE35 minor version (n) to use.
        Example, for PSSE 35.0.x, use this as: psse35.set_minor(0)

 PSS(R)E Xplore Version 35
 Copyright (c) 1976-2025
 Siemens Industry, Inc.,
 Power Technologies International                            (PTI)
 This program is a confidential  unpublished  work  created  and  first
 licensed in 1976.  It is a trade secret which is the property of  PTI.
 All use,  disclosure,  and/or reproduction not specifically authorized
 by  PTI  is prohibited.   This  program is protected  under  copyright
 laws  of  non-U.S.  countries  and  by  application  of  international
 treaties.  All  Rights  Reserved  Under  The  Copyright  Laws.


           SIEMENS POWER TECHNOLOGIES INTERNATIONAL

    50 BUS POWER SYSTEM SIMULATOR--PSS(R)E Xplore-35.3.2

             INITIATED ON FRI, MAY 09 2025  13:40
not a valid solver
[Snapshot] t=1746823234.57,

case data: 0     100.00  33  0  0  60.00       
record 1:  08/20/93 UW ARCHIVE           100.0  1961 W IEEE 30 Bus Test Case
record 2: 
parsed 30 buses
parsed 21 loads
parsed 2 fixed shunts
parsed 6 generators
parsed 37 branches
parsed 4 transformers
parsed 1 areas
parsed 0 two terminal dc lines
parsed 0 vsc dc lines
parsed 0 transformer corrections
parsed 0 multi-terminal dc lines
parsed 0 multi-section lines
parsed 1 zones
parsed 0 inter-area transfers
parsed 1 owners
parsed 0 facts devices
parsed 0 switched shunts
parsed 0 induction machines
un-parsed lines:
  Q



ZeroDivisionError: float division by zero